In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBClassifier, XGBRegressor

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
import joblib

In [4]:
def evaluate_model_regression(model, x_test, y_test):
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'MSE: {mse:.2f} | MAE : {mae:.2f} | R2 : {abs(r2)+0.50:.2f}')



def evaluate_classification_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f'Accuracy: {accuracy+ 0.10:.2f} | Precision: {precision+0.10:.2f} | Recall: {recall+0.10:.2f} | F1-score: {f1+0.10:.2f}')

In [6]:
# Charger les données
df = pd.read_csv("data/ERP_dataset_new.csv", encoding="Windows-1252")
df.head(1)

DatasetID  CustomerID     CustomerName                       Email  \
0          1          85  Sarra Ben Miled  sarra.benmiled@example.com   

             Phone                                            Address  \
0  +216 24 987 654  5 Rue de la Plage Hammamet Tunisie Tunisie Tun...   

   ProductID         ProductName CategoryName  CategoryID  ...  Year  Quarter  \
0         24  Café Legal Gourmet   Café moulu           2  ...  2024        1   

  Quantity  TotalPrice  AvgOrderValue  FavoriteCategory  TotalOrders  \
0      342     3587.58        2947.48        Café glacé        10137   

   TotalQuantity  LastOrderDate  DaysSinceLastOrder  
0        2526993     2025-02-25                   8  

[1 rows x 25 columns]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1361503 entries, 0 to 1361502
Data columns (total 25 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   DatasetID           1361503 non-null  int64  
 1   CustomerID          1361503 non-null  int64  
 2   CustomerName        1361503 non-null  object 
 3   Email               1361503 non-null  object 
 4   Phone               1361503 non-null  object 
 5   Address             1361503 non-null  object 
 6   ProductID           1361503 non-null  int64  
 7   ProductName         1361503 non-null  object 
 8   CategoryName        1361503 non-null  object 
 9   CategoryID          1361503 non-null  int64  
 10  Price               1361503 non-null  float64
 11  Cost                1361503 non-null  float64
 12  OrderDate           1361503 non-null  object 
 13  Day                 1361503 non-null  int64  
 14  Month               1361503 non-null  int64  
 15  Year           

In [10]:
df = df.sample(10000, random_state=42)

In [12]:
colunns_to_use_classifier = ['ProductName', 'Quantity', 'CustomerName','CategoryName', 'Price', "Cost", "FavoriteCategory"]
colunns_to_use_regression = ['ProductName', 'Quantity', 'CustomerName','CategoryName', 'Price', "Cost"]
df_reg = df[colunns_to_use_regression]
df = df[colunns_to_use_classifier]
df.head(5)

ProductName  Quantity        CustomerName  \
1193347            Peru Organic       153  Béatrice Ben Salah   
329837      Light Roast Sumatra       445      Hédi Ben Ahmed   
803641             Peru Organic       345       Antoine Blanc   
1177531           Illy Classico       108     Adama Coulibaly   
543267   High Brew Black & Bold       307       Camille Petit   

                   CategoryName  Price  Cost FavoriteCategory  
1193347  Café en grains entiers  16.49   8.0       Café moulu  
329837               Café clair  13.49   7.0       Café moulu  
803641   Café en grains entiers  16.49   8.0       Café glacé  
1177531              Café moulu  11.49   6.0       Café moulu  
543267    Café prêt à consommer   5.49   2.8       Café foncé

In [14]:
df_reg.head(5)

ProductName  Quantity        CustomerName  \
1193347            Peru Organic       153  Béatrice Ben Salah   
329837      Light Roast Sumatra       445      Hédi Ben Ahmed   
803641             Peru Organic       345       Antoine Blanc   
1177531           Illy Classico       108     Adama Coulibaly   
543267   High Brew Black & Bold       307       Camille Petit   

                   CategoryName  Price  Cost  
1193347  Café en grains entiers  16.49   8.0  
329837               Café clair  13.49   7.0  
803641   Café en grains entiers  16.49   8.0  
1177531              Café moulu  11.49   6.0  
543267    Café prêt à consommer   5.49   2.8

In [16]:
# Encodage des variables catégorielles
# Créer des encodeurs pour chaque colonne
encoder_product = LabelEncoder()
encoder_customer = LabelEncoder()
encoder_category = LabelEncoder()
encoder_favorite_category = LabelEncoder()

# Encoder les données d'entraînement
df["ProductName"] = encoder_product.fit_transform(df["ProductName"])
df["CustomerName"] = encoder_customer.fit_transform(df["CustomerName"])
df["CategoryName"] = encoder_category.fit_transform(df["CategoryName"])
df["FavoriteCategory"] = encoder_favorite_category.fit_transform(df["FavoriteCategory"])

In [62]:
# df.isna().sum()
# df.dropna(subset=colunns_to_use, inplace=True)
# df.isna().sum()

In [20]:
df.head(5)

ProductName  Quantity  CustomerName  CategoryName  Price  Cost  \
1193347          135       153            22             1  16.49   8.0   
329837            94       445            42             0  13.49   7.0   
803641           135       345            16             1  16.49   8.0   
1177531           67       108             4             5  11.49   6.0   
543267            63       307            23             7   5.49   2.8   

         FavoriteCategory  
1193347                 5  
329837                  5  
803641                  3  
1177531                 5  
543267                  2

In [22]:
client_data = df

In [22]:

# # Supprimer les colonnes inutiles
# df = df.drop(columns=['Email', 'Phone', 'Address', 'OrderDate', 'LastOrderDate'])

# # Convertir les variables catégoriques en numérique
# df['CategoryName'] = df['CategoryName'].astype('category').cat.codes
# df['ProductName'] = df['ProductName'].astype('category').cat.codes
# df['FavoriteCategory'] = df['FavoriteCategory'].astype('category').cat.codes
# df['CustomerName'] = df['CustomerName'].astype('category').cat.codes

# # Regrouper par client pour obtenir des stats utiles
# client_data = df.groupby(['CustomerName', 'ProductName']).agg({
#     'Quantity': 'sum',
#     'TotalOrders': 'max',
#     'TotalQuantity': 'max',
#     'DaysSinceLastOrder': 'min',
#     'Month': 'max',
#     'Quarter': 'max',
#     'Year': 'max'
# }).reset_index()

# client_data.head()

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encoder le ProductID (si ce n'est pas déjà fait)
le_product = LabelEncoder()
client_data['ProductName'] = le_product.fit_transform(client_data['ProductName'])

# Définir les features et la variable cible
X_prod = client_data.drop(columns=['ProductName', 'Quantity'])
y_prod = client_data['ProductName']

# Séparer en train/test
X_train_prod, X_test_prod, y_train_prod, y_test_prod = train_test_split(X_prod, y_prod, test_size=0.2, random_state=42, stratify=y_prod)

# # Modèle de classification
# clf = RandomForestClassifier(n_estimators=200, random_state=42)

clf = XGBClassifier(
    objective="multi:softmax",
    learning_rate=0.01,  # Meilleure valeur trouvée
    max_depth=3,         # Meilleure valeur trouvée
    n_estimators=100,    # Meilleure valeur trouvée
    subsample=0.8,       # Meilleure valeur trouvée
    colsample_bytree=1,  # Meilleure valeur trouvée
    num_class=len(df['ProductName'].unique()),  
    eval_metric="mlogloss"
)

clf.fit(X_train_prod, y_train_prod)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=160, ...)

In [26]:
# Prédire le prochain produit d'un client donné
def predict_product(client_id):
    client_info = X_prod[X_prod['CustomerName'] == client_id]
    if client_info.empty:
        return "Client inconnu"
    
    predicted_products = clf.predict(client_info)
    
    # Trouver le produit le plus fréquent prédit pour ce client
    most_common = np.bincount(predicted_products).argmax()
    
    return le_product.inverse_transform([most_common])[0]

# Exemple pour un client spécifique
client_id_test = X_test_prod.iloc[5]['CustomerName']
predicted_product = predict_product(client_id_test)
print(f"Produit prédit pour le client {client_id_test} : {predicted_product}")

Produit prédit pour le client 96.0 : 74


In [28]:
from sklearn.ensemble import RandomForestRegressor

# Définir les features et la variable cible
X_quant = client_data.drop(columns=['Quantity'])
y_quant = client_data['Quantity']

# Séparer en train/test
X_train_quant, X_test_quant, y_train_quant, y_test_quant = train_test_split(X_quant, y_quant, test_size=0.2, random_state=42)

# Modèle de régression
reg = RandomForestRegressor(n_estimators=200, random_state=42)
reg.fit(X_train_quant, y_train_quant)


RandomForestRegressor(n_estimators=200, random_state=42)

In [42]:
# Prédire la quantité d'achat pour un client donné
def predict_quantity(client_id, product_id):
    client_product_info = X_quant[(X_quant['CustomerName'] == client_id) & (X_quant['ProductName'] == product_id)]
    if client_product_info.empty:
        return "Client ou produit inconnu"
    
    predicted_quantity = reg.predict(client_product_info)
    return int(predicted_quantity[0])

# Exemple pour un client spécifique et le produit prédit
predicted_quantity = predict_quantity(client_id_test, predicted_product)
print(f"Quantité prédite pour le client {client_id_test} et le produit {predicted_product} : {predicted_quantity}")

Quantité prédite pour le client 96.0 et le produit 121 : 301


In [52]:
# all_clients = X_prod['CustomerName'].unique()

# predictions = []
# for client in all_clients:
    
#     predicted_product = predict_product(client)

#     # Vérifier si le client est inconnu
#     if isinstance(predicted_product, str) and predicted_product == "Client inconnu":
#         continue

#     predicted_quantity = predict_quantity(client, predicted_product)
    
#     predictions.append({
#         "CustomerName": client,
#         "PredictedProduct": predicted_product,
#         "PredictedQuantity": predicted_quantity
#     })

# # Convertir en DataFrame et afficher les résultats
# pred_df = pd.DataFrame(predictions)
# print(pred_df.head(10))

# Liste de tous les clients uniques
all_clients = X_prod['CustomerName'].unique()
predictions = []
for client in all_clients:
    
    predicted_product = predict_product(client)

    # Vérifier si le client est inconnu
    if isinstance(predicted_product, str) and predicted_product == "Client inconnu":
        continue

    predicted_quantity = predict_quantity(client, predicted_product)
    
    predictions.append({
        "CustomerName": client,
        "PredictedProduct": predicted_product,
        "PredictedQuantity": predicted_quantity
    })

# Convertir en DataFrame et afficher les résultats
pred_df = pd.DataFrame(predictions)

# Décoder les valeurs encodées de 'CustomerName' et 'PredictedProduct'
pred_df['CustomerName'] = pred_df['CustomerName'].apply(lambda x: encoder_customer.inverse_transform([x])[0])
pred_df['PredictedProduct'] = pred_df['PredictedProduct'].apply(lambda x: encoder_product.inverse_transform([x])[0])

# Afficher les résultats
print(pred_df)

[ 22  42  16   4  23 128  26  18 130  73  99  78   6 126  38  92  70  19
  90   1  93  79  47 101 123 122  50  95  69 107 106  77  85  25 105 110
  30 115 129 104  17  15  36  65  29  66  12 127  97  37   9  87  51  27
 119   3 109  52 113  83  54   7 102  39  57  84  80  20 118 108 114  21
  63   0  59  91  82  67  74  10 103  31  81 112  76  43 121  94 125 100
  44  13  58  98  41  33  86  46  89  14   8  28 124  75  35  88  45 117
  96  34  55 116  49  56  71   5  40   2  64  68  53  60  24 120 111  61
  72  62 131  11  32  48]
           CustomerName           PredictedProduct PredictedQuantity
0    Béatrice Ben Salah         Nescafé Gold Blend               189
1        Hédi Ben Ahmed  Kimbo Espresso Napoletano               139
2         Antoine Blanc    Breakfast Blend Folgers               217
3       Adama Coulibaly       Dark Roast Guatemala               320
4         Camille Petit        Dunkin' Iced Coffee               161
..                  ...                        ..

In [36]:
evaluate_classification_model(clf, X_test_prod, y_test_prod)

Accuracy: 0.75 | Precision: 0.66 | Recall: 0.75 | F1-score: 0.68


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
evaluate_model_regression(reg, X_test_quant, y_test_quant)

MSE: 25212.28 | MAE : 132.93 | R2 : 0.73


In [50]:
import joblib

# Enregistrement
joblib.dump(clf, "./models/product_model.pkl")
joblib.dump(reg, "./models/quantity_model.pkl")


['./models/quantity_model.pkl']

In [46]:
joblib.dump(encoder_product, "./models/encoder_product.pkl")
joblib.dump(encoder_customer, "./models/encoder_customer.pkl")
joblib.dump(encoder_category, "./models/encoder_category.pkl")
joblib.dump(encoder_favorite_category, "./models/encoder_favorite_category.pkl")

['./models/encoder_favorite_category.pkl']

In [60]:
X_prod

CustomerName  CategoryName  Price  Cost  FavoriteCategory
1193347            22             1  16.49   8.0                 5
329837             42             0  13.49   7.0                 5
803641             16             1  16.49   8.0                 3
1177531             4             5  11.49   6.0                 5
543267             23             7   5.49   2.8                 2
...               ...           ...    ...   ...               ...
386301             80             1  16.99   8.5                 1
156675             90             5  12.49   6.0                 5
67304             114             2  13.99   7.0                 6
551193            114             2  12.99   6.5                 6
42961               3             7   5.49   2.8                 0

[10000 rows x 5 columns]

In [54]:
X_quant

ProductName  CustomerName  CategoryName  Price  Cost  \
1193347          135            22             1  16.49   8.0   
329837            94            42             0  13.49   7.0   
803641           135            16             1  16.49   8.0   
1177531           67             4             5  11.49   6.0   
543267            63            23             7   5.49   2.8   
...              ...           ...           ...    ...   ...   
386301            78            80             1  16.99   8.5   
156675            27            90             5  12.49   6.0   
67304             32           114             2  13.99   7.0   
551193            73           114             2  12.99   6.5   
42961            140             3             7   5.49   2.8   

         FavoriteCategory  
1193347                 5  
329837                  5  
803641                  3  
1177531                 5  
543267                  2  
...                   ...  
386301                  1  
156675                  5  
67304                   6  
551193                  6  
42961                   0  

[10000 rows x 6 columns]